## Imports

In [ ]:
import datamate
import geopandas as gpd
import pandas as pd
import pictorial

## Load Necessary Configration For Derived Data

In [ ]:
name_derived = "police-data/derived"
key_derived = datamate.config.get_key_for_name(name=name_derived)
path_data_derived = datamate.config.get_path_data_for_key(key=key_derived)

## Load Data

In [ ]:
columns_load = [
    "date",
    "crimeType",
    "longitude",
    "latitude",
    "location",
    "lsoaCode",
    "lsoaName",
    "policeForce",
]

filters = [
    ("year", ">", 2018),
]

In [ ]:
%%time
df = (
    pd.read_parquet(
        path=path_data_derived.joinpath("data.parquet"),
        columns=columns_load,
        filters=filters,
    )
    .assign(
        year=lambda x: x["date"].dt.to_period("Y").dt.to_timestamp(),
        month=lambda x: x["date"].dt.month,
        dayOfMonth=lambda x: x["date"].dt.day,
        count=1,
    )
)

In [ ]:
df.describe(include="category")

category_orders = {
    column: df.groupby(column)["count"].sum().sort_values(ascending=False).index.tolist() 
    for column in df.select_dtypes(include="category")
}
print(f"{category_orders}")

In [ ]:
(
    pictorial.bar(
        df,
        x="year",
        color="policeForce",
        y="count",
        category_orders=category_orders,
    )
    .update_layout(height=600)
)

In [ ]:
(
    pictorial.bar(
        df,
        x="year",
        color="crimeType",
        y="count",
        category_orders=category_orders,
    )
    .update_layout(height=600)
)

In [ ]:
(
    pictorial.bar(
        df,
        x="month",
        y="count",
        color="crimeType",
        category_orders=category_orders,
    )
    .update_layout(height=600)
)

In [ ]:
df_geo = gpd.read_file("https://github.com/gausie/LSOA-2011-GeoJSON/raw/master/lsoa.geojson")

In [ ]:
df_geo